In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

cred = credentials.Certificate('kiosk-firestore-jnsy-bcfe4-firebase-adminsdk-269q4-4a795774bb.json')
firebase_admin.initialize_app(cred)

In [2]:
# 데이터 읽어서 Pandas-DataFrame으로 변환하기
tag = 'cre_keywords'
docs = firestore.client().collection(tag).stream()

doc_list = []
    
for doc in docs:
    doc_list_row = []
    col_name = ['name','keywords']
    
    doc_list_row.append(doc.get('name'))
    doc_list_row.append(doc.get('keywords'))
    
    doc_list.append(doc_list_row)
    
    
doc_list

[['얼그레이 밀크티', ['4300', '주스', '밀크티', '우유', '담백']],
 ['모카 프라페', ['4800', '카페인', '프라페', '우유', '달달', '모카']],
 ['카페모카', ['4500', '카페인', '라떼', '우유', '달달', '모카']],
 ['망고 아이스티', ['3000', '주스', '아이스티', '물', '달달', '과일']],
 ['트리플베리 스무디', ['4800', '주스', '스무디', '얼음', '달달', '과일']],
 ['생강차', ['3300', '주스', '차', '물', '달달', '청량감']],
 ['바닐라 라떼', ['4500', '카페인', '라떼', '우유', '달달', '바닐라']],
 ['돌체라떼', ['3800', '카페인', '라떼', '우유', '달달']],
 ['요거트 스무디', ['4800', '주스', '스무디', '얼음', '달달']],
 ['카페라떼', ['3800', '카페인', '라떼', '우유', '담백']],
 ['민트초코 프라페', ['4800', '카페인', '프라페', '우유', '달달']],
 ['레몬 에이드', ['3800', '주스', '에이드', '탄산', '달달', '과일']],
 ['연유 밀크티', ['4300', '주스', '밀크티', '우유', '달달']],
 ['헤이즐넛 라떼', ['4500', '카페인', '라떼', '우유', '달달']],
 ['딸기 스무디', ['4800', '주스', '스무디', '얼음', '달달', '과일', '딸기']],
 ['복숭아 아이스티', ['3000', '주스', '아이스티', '물', '달달', '과일']],
 ['아메리카노', ['3800', '카페인', '물', '담백']],
 ['카라멜 마끼아또', ['4500', '카페인', '라떼', '우유', '달달']],
 ['레몬 홍차', ['4300', '카페인', '차', '물', '담백', '레몬']],
 ['딸기 에이드', ['3800', '주스', 

In [3]:
# python list [메뉴이름, [키워드]] 형태을 Pandas DataFrame 형태로 변환함
keyword_df = pd.DataFrame(doc_list, columns=col_name)
keyword_df

,name,keywords
0,얼그레이 밀크티,"[4300, 주스, 밀크티, 우유, 담백]"
1,모카 프라페,"[4800, 카페인, 프라페, 우유, 달달, 모카]"
2,카페모카,"[4500, 카페인, 라떼, 우유, 달달, 모카]"
3,망고 아이스티,"[3000, 주스, 아이스티, 물, 달달, 과일]"
4,트리플베리 스무디,"[4800, 주스, 스무디, 얼음, 달달, 과일]"
5,생강차,"[3300, 주스, 차, 물, 달달, 청량감]"
6,바닐라 라떼,"[4500, 카페인, 라떼, 우유, 달달, 바닐라]"
7,돌체라떼,"[3800, 카페인, 라떼, 우유, 달달]"
8,요거트 스무디,"[4800, 주스, 스무디, 얼음, 달달]"
9,카페라떼,"[3800, 카페인, 라떼, 우유, 담백]"


In [4]:
# keywords 리스트 원소들을 공백으로 이어붙임
keyword_df['keywords'] = keyword_df['keywords'].apply(lambda x: (' ').join(x))
keyword_df

,name,keywords
0,얼그레이 밀크티,4300 주스 밀크티 우유 담백
1,모카 프라페,4800 카페인 프라페 우유 달달 모카
2,카페모카,4500 카페인 라떼 우유 달달 모카
3,망고 아이스티,3000 주스 아이스티 물 달달 과일
4,트리플베리 스무디,4800 주스 스무디 얼음 달달 과일
5,생강차,3300 주스 차 물 달달 청량감
6,바닐라 라떼,4500 카페인 라떼 우유 달달 바닐라
7,돌체라떼,3800 카페인 라떼 우유 달달
8,요거트 스무디,4800 주스 스무디 얼음 달달
9,카페라떼,3800 카페인 라떼 우유 담백


In [5]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
keyword_mat = count_vect.fit_transform(keyword_df['keywords']) # 피처 벡터화된 행렬 추출
keyword_mat

from sklearn.metrics.pairwise import cosine_similarity

# 피처 벡터화된 행렬에 코사인 유사도를 적용
keyword_sim = cosine_similarity(keyword_mat, keyword_mat)
keyword_sim[0,:]

# 유사도가 내림차 순으로 정렬된 행렬의 인덱스를 가져온다
keyword_sim_sorted_index = keyword_sim.argsort()[:,::-1]
keyword_sim_sorted_index[:,0] # 모든 행(모든 아이템)에 대해 유사도가 가장 놓은 아이템(첫 열) 인덱스를 추출한다

index_list = keyword_sim_sorted_index[:,1].tolist() # 0은 자기자신, 1은 그 다음 유사도 높은 아이템 인덱스


In [6]:
# 나는 뭘 하고 싶은 거냐면, 최초 DataFrame에서 모든 행을 돌면서, 행 숫자 인덱스로 접근해서, 메뉴이름을 얻고 싶다
# 메뉴 : 유사 메뉴 형태로 출력하고 싶어

# 최초 DataFrame 은 keyword_df.
#keyword_df.iloc[숫자]

for i in range(len(index_list)): # 범위 [0,len(index_list))
    # keyword_df.iloc[i]['name'] # 판다스 접근? .? [컬럼명?] : 컬럼명으로 접근 잘된다
    print(keyword_df.iloc[i]['name'], ':',keyword_df.iloc[index_list[i]]['name']) # 자기 이름이 두 번 나오겠지
    #keyword_df.iloc[index_list[i]]['name']

얼그레이 밀크티 : 연유 밀크티
모카 프라페 : 민트초코 프라페
카페모카 : 헤이즐넛 라떼
망고 아이스티 : 망고 아이스티
트리플베리 스무디 : 트리플베리 스무디
생강차 : 유자차
바닐라 라떼 : 헤이즐넛 라떼
돌체라떼 : 헤이즐넛 라떼
요거트 스무디 : 망고 스무디
카페라떼 : 돌체라떼
민트초코 프라페 : 그린티 프라페
레몬 에이드 : 청귤 에이드
연유 밀크티 : 얼그레이 밀크티
헤이즐넛 라떼 : 카라멜 마끼아또
딸기 스무디 : 트리플베리 스무디
복숭아 아이스티 : 망고 아이스티
아메리카노 : 카페라떼
카라멜 마끼아또 : 카라멜 마끼아또
레몬 홍차 : 아메리카노
딸기 에이드 : 청귤 에이드
오레오 프라페 : 그린티 프라페
청귤 에이드 : 청귤 에이드
레몬차 : 유자차
그린티 프라페 : 그린티 프라페
딸기라떼 : 딸기 에이드
백향과 에이드 : 청귤 에이드
바닐라 프라페 : 민트초코 프라페
망고 스무디 : 트리플베리 스무디
유자차 : 레몬차
